# Clase 3 – Ingesta y Capa Bronce

En esta notebook se inicia la construcción del pipeline de datos meteorológicos, trabajando con los archivos crudos provistos por el SMN.


## 1. Librerías necesarias

In [47]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path


## 2. Configuración de paths y carpetas

In [48]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'

# Crear carpetas si no existen
for path in [BRONCE_DIR]:
    path.mkdir(parents=True, exist_ok=True)


## 3. Lectura del archivo de estaciones

In [74]:
# Ruta del archivo
archivo_estaciones = RAW_DIR / 'estaciones' / 'estaciones_smn.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_estaciones, "r", encoding="latin1") as f:
    lines = f.readlines()[2:]

# Expresión regular para extraer campos:
pattern = re.compile(
    r"^(?P<nombre>.+?)\s{2,}(?P<provincia>.+?)\s{2,}(?P<lat_gr>-?\d+)\s+(?P<lat_min>\d+)\s+(?P<lon_gr>-?\d+)\s+(?P<lon_min>\d+)\s+(?P<altura_m>\d+)\s+(?P<numero>\d+)\s+(?P<numero_oaci>\S+)\s*$"
)

# Extraer los datos
data = []
for line in lines:
    match = pattern.match(line)
    if match:
        data.append(match.groupdict())

# Crear DataFrame
df_estaciones = pd.DataFrame(data)

# Conversión de tipos
df_estaciones[['lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero']] = df_estaciones[[
    'lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero'
]].apply(pd.to_numeric)

print("Estaciones cargadas:", len(df_estaciones))

provincias_unicas = df_estaciones['provincia'].dropna().str.strip().loc[lambda s: s != ''].str.upper().unique()

print("Cantidad de provincias:", len(provincias_unicas))

print("Provincias disponibles:", provincias_unicas)

# Filtrar estaciones de Entre Ríos (.copy va porque evita un warning)
df_ERios = df_estaciones[df_estaciones['provincia'].str.upper().str.strip() == 'ENTRE RIOS'].copy()

# Crear columna coordenadas
df_ERios['coordenadas'] = df_ERios.apply(
    lambda r: f"({int(r['lat_gr'])}.{int(r['lat_min']):02d} , {int(r['lon_gr'])}.{int(r['lon_min']):02d})",
    axis=1
)
df_ERios[['nombre', 'provincia', 'numero', 'numero_oaci','coordenadas']]

Estaciones cargadas: 117
Cantidad de provincias: 25
Provincias disponibles: ['ANTARTIDA' 'BUENOS AIRES' 'CAPITAL FEDERAL' 'CATAMARCA' 'CHACO' 'CHUBUT'
 'CORDOBA' 'CORRIENTES' 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA'
 'LA RIOJA' 'MENDOZA' 'MISIONES' 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN'
 'SAN LUIS' 'SANTA CRUZ' 'SANTA FE' 'SANTIAGO DEL ESTERO'
 'TIERRA DEL FUEGO' 'TUCUMAN']


,nombre,provincia,numero,numero_oaci,coordenadas
57,CONCORDIA AERO,ENTRE RIOS,87395,SAAC,"(-31.18 , -58.00)"
58,GUALEGUAYCHU AERO,ENTRE RIOS,87497,SAAG,"(-33.00 , -58.36)"
59,PARANA AERO,ENTRE RIOS,87374,SAAP,"(-31.47 , -60.29)"


## 4. Selección de estaciones de Entre Ríos

In [50]:
df_ERios = df_estaciones[df_estaciones['provincia'].str.upper() == 'ENTRE RIOS']
df_ERios[['nombre', 'provincia', 'numero', 'numero_oaci']]


,nombre,provincia,numero,numero_oaci
57,CONCORDIA AERO,ENTRE RIOS,87395,SAAC
58,GUALEGUAYCHU AERO,ENTRE RIOS,87497,SAAG
59,PARANA AERO,ENTRE RIOS,87374,SAAP


## 5. Lectura de un archivo horario de ejemplo

In [51]:
archivo_dato = RAW_DIR / 'datohorario' / 'datohorario20240601.txt'

df_dato = pd.read_csv(archivo_dato, sep=';', encoding='latin1')

df_dato.head()

,FECHA HORA TEMP HUM PNM DD FF NOMBRE
0,[HOA] [ºC] [%] [hPa] [gr] [km/hr]
1,01062024 0 14.2 82 1015.7 50 17 AEROPARQUE AERO
2,01062024 1 14.3 80 1015.4 360 9 AEROPARQUE AERO
3,01062024 2 14.1 86 1015.3 360 9 AEROPARQUE AERO
4,01062024 3 14.1 87 1014.8 360 7 AEROPARQUE AERO


## 6. Limpieza básica y detección de nulos

In [52]:
# Contar valores a reemplazar antes de la limpieza
cant_9999_9 = (df_dato == 9999.9).sum().sum()
cant_neg9999 = (df_dato == -9999).sum().sum()

# Reemplazar por NaN
df_dato.replace({9999.9: np.nan, -9999: np.nan}, inplace=True)

# Imprimir resumen
print(f"Reemplazados {cant_9999_9} valores de 9999.9 y {cant_neg9999} valores de -9999 por NaN.")
print("Valores faltantes por columna luego del reemplazo:")
print(df_dato.isna().sum())


Reemplazados 0 valores de 9999.9 y 0 valores de -9999 por NaN.
Valores faltantes por columna luego del reemplazo:
FECHA     HORA  TEMP   HUM   PNM    DD    FF     NOMBRE                                                 0
dtype: int64


## 7. Filtro por estación de Entre Ríos

In [67]:
archivo_dato = RAW_DIR / 'datohorario' / 'datohorario20240601.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_dato, "r", encoding="latin1") as f:
    lines = f.readlines()

# Detectar columnas separadas por múltiples espacios
columnas = re.split(r"\s{2,}", lines[0].strip())

# Leer datos
data = [
    re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
    for line in lines[1:]
    if len(line.strip()) > 0 and not line.isspace()
]

df_dato = pd.DataFrame(data, columns=columnas)

# Filtrar por estaciones de Entre Ríos
df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()

df_nombresEstaciones = df_ERios["nombre"].str.strip().unique()

df_ERios_dia = df_dato[df_dato["NOMBRE"].isin(df_nombresEstaciones)]

# Mostrar todos los resultados (sin limitar con .head())
print(df_ERios_dia.to_string(index=False))


   FECHA HORA TEMP HUM    PNM  DD FF            NOMBRE
01062024    0 16.0  81 1016.5  50 20    CONCORDIA AERO
01062024    1 15.2  87 1016.2  30 17    CONCORDIA AERO
01062024    2 15.2  87 1016.2  30 17    CONCORDIA AERO
01062024    3 15.0  87 1015.6  20 17    CONCORDIA AERO
01062024    4 15.0  89 1015.4  30 15    CONCORDIA AERO
01062024    5 15.0  91 1015.3  90  4    CONCORDIA AERO
01062024    6 14.8  93 1015.3  50 11    CONCORDIA AERO
01062024    7 14.6  91 1015.7  50  9    CONCORDIA AERO
01062024    8 14.6  91 1016.1  30 13    CONCORDIA AERO
01062024    9 15.3  87 1016.4  30  9    CONCORDIA AERO
01062024   10 16.4  85 1016.4  30 15    CONCORDIA AERO
01062024   11 17.6  80 1016.4  20 11    CONCORDIA AERO
01062024   12 19.2  72 1016.0  30 19    CONCORDIA AERO
01062024   13 20.2  70 1014.9  20 19    CONCORDIA AERO
01062024   14 21.4  67 1014.0 360 13    CONCORDIA AERO
01062024   15 22.4  66 1013.4  20 15    CONCORDIA AERO
01062024   16 22.1  67 1013.3  30 13    CONCORDIA AERO
01062024  

## 8. Exportación de archivos filtrados

In [62]:
# Crear carpeta de salida si no existe
BRONCE_DIR.mkdir(parents=True, exist_ok=True)

# Definir la fecha (puede venir del nombre del archivo)
fecha = "20240601"  # o extraela dinámicamente si lo preferís

# Iterar por cada estación de Misiones
for estacion in df_nombresEstaciones:
    nombre_clean = estacion.lower().replace(' ', '_')
    
    # Filtrar las filas de esa estación
    df_estacion = df_ERios_dia[df_ERios_dia["NOMBRE"] == estacion]
    
    # Definir archivos de salida con fecha al inicio
    salida_csv = BRONCE_DIR / f'{fecha}_{nombre_clean}.csv'
    salida_parquet = BRONCE_DIR / f'{fecha}_{nombre_clean}.parquet'
    
    # Exportar
    df_estacion.to_csv(salida_csv, index=False)
    df_estacion.to_parquet(salida_parquet, index=False)
    
    print(f"Exportado: {salida_csv.name} y {salida_parquet.name}")


Exportado: 20240601_concordia_aero.csv y 20240601_concordia_aero.parquet
Exportado: 20240601_gualeguaychu_aero.csv y 20240601_gualeguaychu_aero.parquet
Exportado: 20240601_parana_aero.csv y 20240601_parana_aero.parquet


## 9. Próximos pasos

- Extender este proceso a más días o meses.
- Organizar las salidas por carpeta `/bronce/{estacion}/{año}/`.
- Documentar el diccionario de variables en `metadata/`.


In [70]:
# Por cada archivo, cargar datos y filtrar por las estaciones de Entre Ríos
from glob import glob

errores_globales = 0
nulos_total = 0
reemplazados_9999_9 = 0
reemplazados_neg9999 = 0
detalle_nulos = []

# Buscar todos los archivos datohorario disponibles
archivos_datos = sorted(glob(str(RAW_DIR / 'datohorario' / "datohorario*.txt")))

for archivo in archivos_datos:
    try:
        # Leer el archivo como texto plano
        with open(archivo, encoding="latin1") as f:
            raw_lines = f.readlines()

        # Obtener nombre de columnas desde la primera línea
        header = raw_lines[0].strip()
        columnas = re.split(r"\s{2,}", header)

        # Procesar líneas de datos
        data = [re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
                for line in raw_lines[1:] if len(line.strip()) > 0 and not line.isspace()]

        # Crear DataFrame
        df_dato = pd.DataFrame(data, columns=columnas)
        df_dato.columns = df_dato.columns.str.strip()
        df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()
        
        # Convertir columnas numéricas si es posible
        for col in df_dato.columns:
            try:
                df_dato[col] = pd.to_numeric(df_dato[col])
            except:
                pass

        # Reemplazo
        df_dato.replace({9999.9: np.nan, -9999: np.nan}, inplace=True)

        # Contar nulos
        nulos_total += df_dato.isna().sum().sum()

        # Filtrar por estaciones de EntreRios
        df_dato_eRios = df_dato[df_dato["NOMBRE"].str.strip().isin(df_nombresEstaciones)]

        # Obtener fecha
        fecha = Path(archivo).stem.replace("datohorario", "")

                # Guardar archivos por estación + generar detalle
        for nombre in df_nombresEstaciones:
            nombre_clean = nombre.lower().replace(" ", "_")
            df_estacion = df_misiones[df_misiones["NOMBRE"] == nombre]

            if not df_estacion.empty:
                path_estacion = BRONCE_DIR / nombre_clean
                path_estacion.mkdir(parents=True, exist_ok=True)

                df_estacion.to_parquet(path_estacion / f"{fecha}.parquet", index=False)
                df_estacion.to_csv(path_estacion / f"{fecha}.csv", index=False)

                # Registro de nulos por estación
                nulos_por_col = df_estacion.isna().sum()
                detalle_nulos.append({
                    "archivo": Path(archivo).name,
                    "fecha": fecha,
                    "estacion": nombre,
                    "nulos_totales": int(nulos_por_col.sum()),
                    "nulos_por_columna": json.dumps(nulos_por_col[nulos_por_col > 0].to_dict())
                })
    except Exception as e:
        errores_globales += 1
        continue
    
    
# Reporte final
print("✅ Proceso completado.")
print(f"Días procesados: {len(archivos_datos)}")
print(f"Errores al procesar archivos: {errores_globales}")
print(f"Valores reemplazados: {reemplazados_9999_9} de 9999.9 y {reemplazados_neg9999} de -9999")
print(f"Total de valores nulos luego de limpieza: {nulos_total}")

# Guardar resumen general
reporte = {
    "dias_procesados": [len(archivos_datos)],
    "errores": [errores_globales],
    "reemplazados_9999_9": [reemplazados_9999_9],
    "reemplazados_-9999": [reemplazados_neg9999],
    "valores_nulos_totales": [nulos_total],
}
df_reporte = pd.DataFrame(reporte)
df_reporte.to_csv(BRONCE_DIR / "reporte_resumen.csv", index=False)

# Guardar detalle de nulos por archivo y estación
df_detalle = pd.DataFrame(detalle_nulos)
df_detalle.to_csv(BRONCE_DIR / "reporte_nulos_detalle.csv", index=False)


✅ Proceso completado.
Días procesados: 391
Errores al procesar archivos: 391
Valores reemplazados: 0 de 9999.9 y 0 de -9999
Total de valores nulos luego de limpieza: 260790
